In [1]:
%load_ext cython

In [2]:
from galois.shmem import *
from galois.graphs import *

In [3]:
print(setActiveThreads(16))

16


In [4]:
%%cython
# distutils: extra_link_args=["-lgalois_shmem"]
from galois.shmem cimport *
from galois.shmem import *
from cython.operator cimport preincrement, dereference as deref

ctypedef atomic[uint32_t] atomuint32_t
ctypedef atomic[uint64_t] atomuint64_t
##############################################################################
## Pagerank implementation
###############################################################################
#
# Struct for Pagerank
#
cdef struct NodeTy:
    float rank
    uint32_t nout

ctypedef LC_CSR_Graph[NodeTy, void, dummy_true] Graph

# Cython bug: using a nested class from a previous typedef doesn't
# work for the time being. Instead, the full template specialization
# must be used to get the member type.
ctypedef LC_CSR_Graph[NodeTy, void, dummy_true].GraphNode GNode

#
# Constants for Pagerank
#
cdef float ALPHA = 0.85
cdef float INIT_RESIDUAL = 1 - ALPHA;
cdef float TOLERANCE   = 1.0e-3;
cdef uint32_t MAX_ITER = 1000;

#
# Initialization for Pagerank
#
cdef void InitializePR(Graph *g):
    cdef unsigned long numNodes = g[0].size()
    cdef NodeTy *data
    gPrint(b"Number of nodes : ", numNodes, b"\n")
    for n in range(numNodes):
        #gPrint(n,"\n")
        data = &g[0].getData(n)
        data[0].rank = INIT_RESIDUAL
        data[0].nout = 0

#
# Operator for computing outdegree of nodes in the Graph
#
cdef void computeOutDeg_operator(Graph *g, LargeArray[atomuint64_t] *largeArray, GNode n) nogil:
    cdef:
        GNode dst

    edges = g.edges(n)
    for ii in edges:
        dst = g.getEdgeDst(ii)
        largeArray[0][<size_t>dst].fetch_add(1)

#
# Operator for assigning outdegree of nodes in the Graph
#
cdef void assignOutDeg_operator(Graph *g, LargeArray[atomuint64_t] *largeArray, GNode n) nogil:
    cdef NodeTy *src_data

    src_data = &g.getData(n)
    src_data.nout = largeArray[0][<size_t>n].load()

#
# Main callsite for computing outdegree of nodes in the Graph
#
cdef void computeOutDeg(Graph *graph):
    cdef:
        uint64_t numNodes = graph[0].size()
        LargeArray[atomuint64_t] largeArray

    largeArray.allocateInterleaved(numNodes)
    with nogil:
        do_all(iterate(graph.begin(), graph.end()),
                        bind_leading(&computeOutDeg_operator, graph, &largeArray), steal(),
                        loopname("ComputeDegree"))

        do_all(iterate(graph.begin(), graph.end()),
                        bind_leading(&assignOutDeg_operator, graph, &largeArray))


#
# Operator for PageRank
#
cdef void pagerankPullTopo_operator(Graph *g, GReduceMax[float] *max_delta, GNode n) nogil:
    cdef:
        GNode dst
        NodeTy *dst_data
        NodeTy *src_data
        float sum = 0
        float value = 0
        float diff = 0;
    src_data = &g.getData(n)
    edges = g.edges(n, FLAG_UNPROTECTED)
    for ii in edges:
        dst_data = &g.getData(g.getEdgeDst(ii), FLAG_UNPROTECTED)
        sum += dst_data[0].rank / dst_data[0].nout
    value = sum * ALPHA + (1.0 - ALPHA)
    diff = fabs(value - src_data[0].rank);
    src_data[0].rank = value
    max_delta[0].update(diff)

#
# Pagerank routine: Loop till convergence
#
cdef void pagerankPullTopo(Graph *graph, uint32_t max_iterations):
    cdef:
        GReduceMax[float] max_delta
        float delta = 0
        uint32_t iteration = 0
        Timer T

    T.start()
    while(1):
        with nogil:
            do_all(iterate(graph.begin(), graph.end()),
                        bind_leading(&pagerankPullTopo_operator, graph, &max_delta), steal(),
                        loopname("PageRank"))

        delta = max_delta.reduce()
        iteration += 1
        if(delta <= TOLERANCE or iteration >= max_iterations):
            break
        max_delta.reset();

    T.stop()
    print("Elapsed time:", T.get(), " milliseconds.")
    if(iteration >= max_iterations):
        print("WARNING : failed to converge in ", iteration, " iterations")


#
# Main callsite for Pagerank
#
def pagerank(uint32_t max_iterations, filename):
    cdef Graph graph

    ## Read the CSR format of graph
    ## directly from disk.
    graph.readGraphFromGRFile(bytes(filename, "utf-8"))

    InitializePR(&graph)
    computeOutDeg(&graph)
    pagerankPullTopo(&graph, max_iterations)
    #printValuePR(&graph)
    return [graph.getData(i).rank for i in range(graph.size())]

In [5]:
results = pagerank(10, "/home/amp/Downloads/indochina-2004.csgr")

Elapsed time: 618  milliseconds.


In [6]:
print(results[:10], len(results))

[1.446841835975647, 0.3037269115447998, 0.6015442609786987, 0.4101789593696594, 0.6832333207130432, 0.45934468507766724, 0.42385345697402954, 0.9236875772476196, 0.37441474199295044, 5.855180740356445] 7414866
